In [41]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import getpass
import pandas as pd
import numpy as np


client_id = getpass.getpass()
client_secret = getpass.getpass()



sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id,
                                               client_secret,
                                               redirect_uri="https://localhost:8089/callback",
                                               scope="user-library-read"))



results = sp.current_user_saved_tracks() ## here we don't use endpoints (urls basically), and just use sp. and then there are methods



In [42]:
spain = pd.read_csv('spain_summer_songs.csv')

In [56]:
import scipy.stats as st

spain_feat = spain[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'liveness', 'valence', 'tempo']]
conf_intervals = pd.DataFrame(columns=['trait', 'bottom', 'top'])


traits = ['danceability', 'energy', 'loudness', 'liveness', 'valence', 'tempo']
n= len(spain_feat)

cibot = []
citop = []
for feat in traits:
    mean_ = spain_feat[feat].mean()
    s = spain_feat[feat].std(ddof=0)
    print(feat)
    cibot.append(st.norm.interval(0.95, loc=mean_, scale=s/np.sqrt(n))[0])
    citop.append(st.norm.interval(0.95, loc=mean_, scale=s/np.sqrt(n))[1])
print(citop)
print(cibot)

conf_intervals['trait'] = traits
conf_intervals['bottom'] = cibot
conf_intervals['top'] = citop

conf_intervals.to_csv('confidence_intervals.csv')

danceability
energy
loudness
liveness
valence
tempo
[0.7782801797502868, 0.8739534069249884, -5.4469988978542085, 0.22214699667020965, 0.8787460500560335, 134.81486842443496]
[0.7039698202497134, 0.7857965930750119, -7.537688602145791, 0.10234675332979039, 0.7868164499439664, 118.13281907556507]


In [58]:
conf_intervals.set_index('trait',inplace= True)
conf_intervals

,bottom,top
trait,,
danceability,0.703970,0.778280
energy,0.785797,0.873953
loudness,-7.537689,-5.446999
liveness,0.102347,0.222147
valence,0.786816,0.878746
tempo,118.132819,134.814868


In [60]:
conf_intervals.to_csv('confidence_intervals.csv')

In [82]:
def hit_detector(row):

    hit_score = 0
    needs = np.array([0, 0, 0, 0])

    traits = ['danceability', 'energy', 'valence']


    for trait in traits:
        
    

        if row[trait] >= conf_intervals.loc[trait, 'bottom']: # (abs(row[trait]) < abs(conf_intervals['top'][traits.index(trait)])):


            hit_score += 1
            
            #print(trait, row[trait])
        
        else:
            hit_score += 0

            if trait == 'danceability':
                needs += np.array([0,1,0,0])
            elif trait == 'energy':
                needs += np.array([0,0,1,0])
            elif trait == 'valence':
                needs += np.array([0,0,0,1])
                
            #needs.append(trait)
            #print(trait,conf_intervals['bottom'][traits.index(trait)],conf_intervals['top'][traits.index(trait)])

            #print(trait, row[trait])

    if row['mode'] == 1:
        hit_score += 0.5
    else:
        hit_score += 0


    if hit_score >= 3:
        #print(needs)

        return (1, 0,0,0)

    else:
        
        return needs

In [61]:
def playlist_dataframe(playlist_link):

    playlist_URI = playlist_link.split("/")[-1].split("?")[0]

    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]] 

    df = pd.DataFrame(columns=['artist_name','track_name','release_date','danceability',
    'energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url',
    'duration_ms','time_signature'])


    for track in sp.playlist_tracks(playlist_URI)["items"]:

        info_vector = []
        #URI
        track_uri = track["track"]["uri"]
        
        #Track name
        track_name = track["track"]["name"]
        
        #Name, popularity, genre
        artist_name = track["track"]["artists"][0]["name"]
      
        #Release Date
        release_date = track['track']['album']['release_date'] ## keep full date

        info_vector = [artist_name,track_name,release_date]

        info_vector.extend(list((sp.audio_features(track_uri)[0].values())))

        df.loc[len(df)] = info_vector

        df[['Hit', 'Dance_miss', 'Energy_miss', 'Valence_miss']] = df.apply(hit_detector,axis = 1, result_type='expand')

    return df

In [89]:

def playlist_dataframe(playlist_link):

    playlist_URI = playlist_link.split("/")[-1].split("?")[0]

    track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]] 

    df = pd.DataFrame(columns=['artist_name','track_name','album_name','release_date','artist_popularity','track_popularity', 'artist_genres','danceability',
    'energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','type','id','uri','track_href','analysis_url',
    'duration_ms','time_signature'])



    for track in sp.playlist_tracks(playlist_URI)["items"]:

        info_vector = []
        #URI
        track_uri = track["track"]["uri"]
        
        #Track name
        track_name = track["track"]["name"]
        
        #Main Artist
        artist_uri = track["track"]["artists"][0]["uri"]
        artist_info = sp.artist(artist_uri)
        
        #Name, popularity, genre
        artist_name = track["track"]["artists"][0]["name"]
        artist_pop = artist_info["popularity"]
        artist_genres = artist_info["genres"]
        
        #Album
        album = track["track"]["album"]["name"]
        #Release Date
        release_date = track['track']['album']['release_date'] ## keep full date 

        #Popularity of the track
        track_pop = track["track"]["popularity"]

        info_vector = [artist_name,track_name,album,release_date,artist_pop,track_pop,artist_genres]

        info_vector.extend(list((sp.audio_features(track_uri)[0].values())))


        df.loc[len(df)] = info_vector

    df[['Hit', 'Dance_miss', 'Energy_miss', 'Valence_miss']] = df.apply(hit_detector,axis = 1, result_type='expand')

    return df

In [90]:
playlist_dataframe('https://open.spotify.com/playlist/2vo9KPbGNMyxR6QYZif97Z?si=d0e83e5825ce497d')

,artist_name,track_name,album_name,release_date,artist_popularity,track_popularity,artist_genres,danceability,energy,key,...,id,uri,track_href,analysis_url,duration_ms,time_signature,Hit,Dance_miss,Energy_miss,Valence_miss
0,Barry Adamson,Something Wicked This Way Comes,Oedipus Schmoedipus,1993-07-01,32,40,[british alternative rock],0.689,0.465,10,...,4FpeO73o95yo9WjoA7azaV,spotify:track:4FpeO73o95yo9WjoA7azaV,https://api.spotify.com/v1/tracks/4FpeO73o95yo...,https://api.spotify.com/v1/audio-analysis/4Fpe...,273333,4,0,1,1,1
1,Jon Kennedy,Brown Acid,Take My Drum to England,2003,46,17,"[electronica, ninja, nu jazz, trip hop, turnta...",0.752,0.730,9,...,2rM2d10FiJt7e4y9f34xfl,spotify:track:2rM2d10FiJt7e4y9f34xfl,https://api.spotify.com/v1/tracks/2rM2d10FiJt7...,https://api.spotify.com/v1/audio-analysis/2rM2...,250427,4,0,0,1,1
2,Jon Kennedy,The Loafer,Take My Drum to England,2003,46,11,"[electronica, ninja, nu jazz, trip hop, turnta...",0.705,0.577,6,...,2N60bQm6I3DTx9bjvJSRum,spotify:track:2N60bQm6I3DTx9bjvJSRum,https://api.spotify.com/v1/tracks/2N60bQm6I3DT...,https://api.spotify.com/v1/audio-analysis/2N60...,278570,4,0,0,1,1
3,Moodymann,Hold It Down,Moodymann,2014-01-27,45,29,"[deep house, detroit house, detroit techno, el...",0.757,0.317,11,...,0HQZJbulEJMr5hnGdHz8Wu,spotify:track:0HQZJbulEJMr5hnGdHz8Wu,https://api.spotify.com/v1/tracks/0HQZJbulEJMr...,https://api.spotify.com/v1/audio-analysis/0HQZ...,174027,4,0,0,1,0
4,Moodymann,Sloppy Cosmic,Moodymann,2014-01-27,45,33,"[deep house, detroit house, detroit techno, el...",0.728,0.423,6,...,2E8EbnfOycV8BaUzbCsGqa,spotify:track:2E8EbnfOycV8BaUzbCsGqa,https://api.spotify.com/v1/tracks/2E8EbnfOycV8...,https://api.spotify.com/v1/audio-analysis/2E8E...,708707,4,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Cymande,Brothers On The Slide,Promised Heights,1974-01-01,47,56,"[afrobeat, funk, psychedelic soul, soul, world]",0.815,0.602,7,...,3ObBmm3X92aff7Dct4drkj,spotify:track:3ObBmm3X92aff7Dct4drkj,https://api.spotify.com/v1/tracks/3ObBmm3X92af...,https://api.spotify.com/v1/audio-analysis/3ObB...,251400,4,0,0,1,0
67,The Meters,Cissy Strut,The Meters,1969,51,61,"[classic soul, funk, instrumental soul, new or...",0.507,0.763,0,...,0WSlOSMLJWoWUpWci9nnRb,spotify:track:0WSlOSMLJWoWUpWci9nnRb,https://api.spotify.com/v1/tracks/0WSlOSMLJWoW...,https://api.spotify.com/v1/audio-analysis/0WSl...,186853,4,0,1,1,1
68,The Meters,Here Comes the Meter Man,The Meters,1969,51,38,"[classic soul, funk, instrumental soul, new or...",0.569,0.888,0,...,1mvsBRtgjW7sxKRNguQtz4,spotify:track:1mvsBRtgjW7sxKRNguQtz4,https://api.spotify.com/v1/tracks/1mvsBRtgjW7s...,https://api.spotify.com/v1/audio-analysis/1mvs...,175200,4,0,1,0,0
69,The Meters,Cardova,The Meters,1969,51,39,"[classic soul, funk, instrumental soul, new or...",0.742,0.692,5,...,0h2xiysQLgkfu6S8U5tBkN,spotify:track:0h2xiysQLgkfu6S8U5tBkN,https://api.spotify.com/v1/tracks/0h2xiysQLgkf...,https://api.spotify.com/v1/audio-analysis/0h2x...,275253,4,0,0,1,1
